In [1]:
df = pd.read_excel("techstore sales_data.xlsx")
df.head()


NameError: name 'pd' is not defined

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_excel("techstore sales_data.xlsx")
df.head()


,Transaction ID,Product,Quantity,Unit Price (INR),Month,Sales Date,Customer Name,City,State,Country,Sales Channel,Payment Type,Total Sale (INR),Profit
0,TX0001,Smartwatch,9,8941,Apr,2024-04-12,Vivaan Sastry,Bharatpur,Arunachal Pradesh,India,Store,Credit Card,80469,24140.7
1,TX0002,Desktop PC,9,19827,Feb,2024-02-06,Ishaan Madan,Nellore,Odisha,India,Store,Credit Card,178443,53532.9
2,TX0003,Desktop PC,4,16640,Dec,2023-12-16,Azad Sandhu,New Delhi,Bihar,India,Online,Credit Card,66560,19968.0
3,TX0004,E-Reader,3,2945,Sep,2024-09-11,Shray Chawla,Gandhidham,Manipur,India,Online,Credit Card,8835,2650.5
4,TX0005,Laptop,4,2848,Oct,2023-10-24,Zara Kata,Surendranagar Dudhrej,Madhya Pradesh,India,Direct,Net Banking,11392,3417.6


In [4]:
df.columns


Index(['Transaction ID', 'Product', 'Quantity', 'Unit Price (INR)', 'Month',
       'Sales Date', 'Customer Name', 'City', 'State', 'Country',
       'Sales Channel', 'Payment Type', 'Total Sale (INR)', 'Profit'],
      dtype='object')

In [5]:
df['Sales Date'] = pd.to_datetime(df['Sales Date'])


In [6]:
reference_date = df['Sales Date'].max()
reference_date


Timestamp('2024-09-23 00:00:00')

In [7]:
rfm = df.groupby('Customer Name').agg({
    'Sales Date': lambda x: (reference_date - x.max()).days,
    'Transaction ID': 'count',
    'Total Sale (INR)': 'sum'
})


In [8]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()


,Recency,Frequency,Monetary
Customer Name,,,
Aaina Biswas,273,1,125468
Aaina Dayal,277,1,57224
Aaina Dhingra,3,1,54945
Aaina Kapadia,10,1,5768
Aaina Kota,1,1,5204


In [9]:
rfm.describe()


,Recency,Frequency,Monetary
count,996.000000,996.000000,996.000000
mean,178.321285,1.004016,51407.747992
std,106.440524,0.063277,39294.918484
min,0.000000,1.000000,2043.000000
25%,86.000000,1.000000,20197.250000
50%,181.000000,1.000000,39940.000000
75%,270.250000,1.000000,74091.000000
max,365.000000,2.000000,257929.000000


In [10]:
rfm.sort_values('Monetary', ascending=False).head(10)


,Recency,Frequency,Monetary
Customer Name,,,
Arhaan Balay,105,2,257929
Ishaan Madan,230,1,178443
Prisha Bakshi,12,1,177660
Pari Koshy,158,1,173988
Mehul Ravi,170,1,173835
Arnav Sarraf,162,1,172854
Jhanvi Kata,296,1,168246
Mishti Sarraf,9,1,166455
Anahi Wagle,103,1,162666


In [11]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'], 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])


ValueError: Bin edges must be unique: Index([1.0, 1.0, 1.0, 1.0, 1.0, 2.0], dtype='float64', name='Frequency').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [12]:
rfm['F_Score'] = pd.qcut(
    rfm['Frequency'],
    5,
    labels=[1,2,3,4,5],
    duplicates='drop'
)


ValueError: Bin labels must be one fewer than the number of bin edges

In [13]:
# Recency score (lower recency = better score)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])

# Frequency score (handles duplicate values safely)
rfm['F_Score'] = pd.qcut(
    rfm['Frequency'],
    q=rfm['Frequency'].nunique(),
    labels=False,
    duplicates='drop'
) + 1

# Monetary score (higher spending = better score)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

# Combine RFM score
rfm['RFM_Score'] = (
    rfm['R_Score'].astype(str) +
    rfm['F_Score'].astype(str) +
    rfm['M_Score'].astype(str)
)

rfm.head()


,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
Customer Name,,,,,,,
Aaina Biswas,273,1,125468,2,1,5,215
Aaina Dayal,277,1,57224,2,1,4,214
Aaina Dhingra,3,1,54945,5,1,4,514
Aaina Kapadia,10,1,5768,5,1,1,511
Aaina Kota,1,1,5204,5,1,1,511
